In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display
import sys
sys.path.append('../scripts') 
from functions import *
from config import nutrient_info, conversion_factors

In [2]:
df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_15608\2711278687.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')


In [3]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [4]:
df_butter_margarine_spreads = df_food_scaled[df_food_scaled['food_group'] == 'butter_margarine_spreads']
df_butter_margarine_spreads.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
290425,Délice : extra Light,3.777249,1.079267,52.906567
301075,le Parfait Wurziger Brotaufstrich,3.147747,2.552177,71.926246
305905,Becel: Pro Activ,2.592644,0.929681,45.535122
172217,Margarine,2.445222,0.994892,45.603756
284495,Coop Qualité & Prix - Margarine,1.305216,1.314421,40.631510
...,...,...,...,...
161878,Vegetable fat,0.000000,0.748403,0.000000
298635,aha - KOKOS Margarine,0.000000,1.277303,0.000000
297998,Natur Kraft Werke - Kürbiskernmus,0.000000,0.749318,0.000000
311133,Coop Naturaplan - Bio Butter,0.000000,0.360202,0.000000


In [5]:
high_qi_butter_margarine_spreads = df_butter_margarine_spreads[df_butter_margarine_spreads['QI'] > 1].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [6]:
high_qi_butter_margarine_spreads[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
290425,Délice : extra Light,5.0,3.777249,1.079267,52.906567
301075,le Parfait Wurziger Brotaufstrich,25.0,3.147747,2.552177,71.926246
305905,Becel: Pro Activ,10.0,2.592644,0.929681,45.535122
172217,Margarine,5.0,2.445222,0.994892,45.603756
284495,Coop Qualité & Prix - Margarine,10.0,1.305216,1.314421,40.631510
284129,Volg - Margarine,10.0,1.270157,0.516791,40.688617
84402,Sesame paste,10.0,1.223884,0.601614,61.104151
277746,MIGROS Leger FIT-ONNAISE CLASSIC,7.0,1.188645,1.517040,28.165576
277730,Nestlé THOMY Thomynaise light 60% weniger Fett,20.0,1.117862,1.695834,26.322371
284854,M Budget - Margarine,10.0,1.063092,1.361600,38.315429


In [7]:
ratio_cols = [i for i in high_qi_butter_margarine_spreads.columns if '_ratio_scaled' in i]

df_corr = high_qi_butter_margarine_spreads[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

calcium_eaten_ratio_scaled                                  -0.230954
magnesium_eaten_ratio_scaled                                -0.223934
vitamin_e_activity_eaten_ratio_scaled                       -0.089371
cholesterol_eaten_ratio_scaled                              -0.066013
phosphorus_eaten_ratio_scaled                               -0.003088
protein_eaten_ratio_scaled                                   0.059167
fat_eaten_ratio_scaled                                       0.063165
potassium_eaten_ratio_scaled                                 0.083161
salt_eaten_ratio_scaled                                      0.094208
iron_eaten_ratio_scaled                                      0.137926
zinc_eaten_ratio_scaled                                      0.222515
fatty_acids_saturated_eaten_ratio_scaled                     0.239290
niacin_eaten_ratio_scaled                                    0.263989
fiber_eaten_ratio_scaled                                     0.317678
sodium_eaten_ratio_s

In [8]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_butter_margarine_spreads, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )

    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_butter_margarine_spreads, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [9]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- folate
- vitamin b1
- RAE
- vitamin b12
- vitamin b6
- vitamin d
- vitamin e

In [10]:
exclude_list = ['vitamin_b1_eaten', 'vitamin_b6_eaten', 'folate_eaten', 'vitamin_b12_eaten', 'vitamin_d_eaten', 'vitamin_e_eaten', 'all_trans_retinol_equivalents_activity_eaten']
qualifying_nutrients = list(nutrient_info.keys())

for idx, row in high_qi_butter_margarine_spreads.iterrows():

    qi_incl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=[])
    qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    nb_incl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=[]) 
    nb_excl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    print(f"Food item : {row['combined_name']}")
    print(f"QI (incl. all): {qi_incl}")
    print(f"QI (excl. {', '.join(exclude_list)}): {qi_excl}")
    print(f"NB (incl. all): {nb_incl:.2f}")
    print(f"NB (excl. {', '.join(exclude_list)}): {nb_excl:.2f}")
    print("-----------")

Food item : Délice : extra Light
QI (incl. all): 3.280823395603766
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 1.10790069394361
NB (incl. all): 55.37
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 41.28
-----------
Food item : le Parfait Wurziger Brotaufstrich
QI (incl. all): 3.126093147235559
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 1.9864396863234262
NB (incl. all): 77.13
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, folate_eaten, vitamin_b12_eaten, vitamin_d_eaten, vitamin_e_eaten, all_trans_retinol_equivalents_activity_eaten): 78.05
-----------
Food item : Becel: Pro Activ
QI (incl. all): 2.2972389179664696
QI (excl. vitamin_b1_eaten, vitam

In [11]:
df_butter_margarine_spreads[df_butter_margarine_spreads['combined_name'] == 'Cenovis, yeast spread'].drop_duplicates('combined_name')[[i for i in df_butter_margarine_spreads if '_ratio_scaled' in i]].T

,247590
folate_eaten_ratio_scaled,0.000000
niacin_eaten_ratio_scaled,0.000000
pantothenic_acid_eaten_ratio_scaled,0.000000
vitamin_b2_eaten_ratio_scaled,0.000000
vitamin_b1_eaten_ratio_scaled,0.000000
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.000000
vitamin_b12_eaten_ratio_scaled,0.000000
vitamin_b6_eaten_ratio_scaled,0.000000
vitamin_c_eaten_ratio_scaled,0.000000
